# Randomized Benchmarking

In [2]:
import numpy as np
from qutip import tensor, basis, ket2dm, fidelity
from qutip_qip.circuit import QubitCircuit
from qutip_qip.operations.gates import rz
from chalmers_qubit.sarimner import (
    SarimnerProcessor,
    SarimnerModel,
    SarimnerCompiler,
    DecoherenceNoise,
)
from chalmers_qubit.base.operations import project_on_qubit

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Single-qubit Clifford Randomized Benchmarking

Setup device

In [5]:
transmon_dict = {
    0: {"frequency": 5.02, "anharmonicity": -0.3},
}
decoherence_dict = {
    0: {"t1": 60e3, "t2": 100e3},
}
# Set up the model, compiler, and processor
model = SarimnerModel(transmon_dict=transmon_dict)
compiler = SarimnerCompiler(model=model)
noise = [DecoherenceNoise(decoherence_dict=decoherence_dict)]
sarimner = SarimnerProcessor(model=model, compiler=compiler, noise=noise)

## Create Single-qubit Clifford Group 

We now create the single-qubit Clifford group, using the XY-decomposition

## Simulate over sequence lengths

In [ ]:
import random

num_cliffords = 24

# loop over seqneuce lengths
# sequence_lengths = [8, 16, 32, 64, 128, 256, 512, 1024]
sequence_list = [8]


decay_rate = []
# Loop over sequence lengths
for sequence_length in sequence_list:
    circuit = QubitCircuit(1) # Initialize qubit circuit
    # Sample number of Cliffords equal to the sequence length
    for m in range(sequence_length):
        # Sample a Clifford from the Clifford group
        clifford_idx = random.randint(0, num_cliffords)
        # Add the physical decomposition of the Clifford to the circuit
        circuit.add_gate("RX", targets=0, arg_value=np.pi)
        
    # Add the inverse Clifford gate to the circuit
    # ...
    
    # Simulate the quantum circuit and compute the probability of remaning in state |0>
    initial_state = basis(3,0)
    result = sarimner.run_state(init_state=initial_state, 
                                qc=circuit,
                                e_ops=[ket2dm(basis(3,0))])
    p0 = result.expect[0][-1]
    decay_rate.append(p0)
print(decay_rate)

[0.9966136094208787]


In [ ]:
from qutip_qip.circuit import QubitCircuit
import random

num_cliffords = 24

# loop over seqneuce lengths
# sequence_lengths = [8, 16, 32, 64, 128, 256, 512, 1024]
sequence_list = [8]


decay_rate = []
# Loop over sequence lengths
for sequence_length in sequence_list:
    circuit = QubitCircuit(1)  # Initialize qubit circuit
    # Sample number of Cliffords equal to the sequence length
    for m in range(sequence_length):
        # Sample a Clifford element from the Clifford group
        
        # Add the corresponding physical decomposition of the Clifford element to the circuit

    
    # Find and add the physical decomposition of the inverse Clifford element.
    # ...

    # Simulate the quantum circuit and compute the probability of remaning in state |0>
    initial_state = basis(3, 0)
    result = sarimner.run_state(
        init_state=initial_state, qc=circuit, e_ops=[ket2dm(basis(3, 0))]
    )
    p0 = result.expect[0][-1]
    decay_rate.append(p0)
print(decay_rate)